In [1]:
from transformers import GPT2TokenizerFast, GPT2LMHeadModel

tokenizer = GPT2TokenizerFast.from_pretrained('gpt2-large')
model = GPT2LMHeadModel.from_pretrained('gpt2-large').cuda()

In [28]:
tokenized = tokenizer.batch_encode_plus(['Who is Barack Obama? The man who has been'], return_tensors='pt')
output = model(
    input_ids=tokenized['input_ids'].cuda(),
    attention_mask=tokenized['attention_mask'].cuda(),
    output_hidden_states=True,
    output_attentions=True)

In [29]:
tokenized = tokenizer('president', return_tensors='pt')['input_ids'].cuda()
president = model.transformer.wte(tokenized)[0]
import torch
for i, layer in enumerate(output.hidden_states):
    diff = -1
    if i > 0:
        # diff = torch.cosine_similarity(layer[:,-1,:], output.hidden_states[i - 1][:,-1,:], dim=1).item()
        diff = torch.cosine_similarity(layer[:,-1,:], model.transformer.wte.weight, dim=1)
        diff = tokenizer.decode(torch.topk(diff, 10).indices)
        # diff = torch.norm(layer[:,-1,:], dim=1).item()
    layer = model.lm_head(layer)
    print(i, tokenizer.decode(layer.argmax(dim=-1)[0][-1]), diff)

    # layer = output.attentions[i]
    # if i > 0:
    #     # diff = torch.cosine_similarity(layer[:,-1,:], output.hidden_states[i - 1][:,-1,:], dim=1).item()
    #     diff = torch.cosine_similarity(layer[:,-1,:], model.transformer.wte.weight, dim=1)
    #     diff = tokenizer.decode(torch.topk(diff, 20).indices)
    #     # diff = torch.norm(layer[:,-1,:], dim=1).item()
    # layer = model.lm_head(layer)
    # print(i, tokenizer.decode(layer.argmax(dim=-1)[0][-1]), diff)

0  been -1
1  replaced  been taken able seen replaced getting found considered made living
2  effected  able seen taken getting been made living going found extremely
3  rescued  able taken seen made rescued removed replaced treated documented exposed
4  rescued  able seen taken removed rescued made exposed treated been going
5  rescued  able seen taken made removed been rescued going documented considered
6  rescued  able seen taken made been going removed documented rescued living
7  rescued  able seen taken made been rescued removed born going studied
8  rescued  able taken seen made been going described rescued attacked used
9  seen  seen able taken made attacked talked used described called been
10  seen  seen able made taken described talked called attacked used been
11  seen  seen made able taken talked described used attacked called going
12  rescued  made seen taken able talked described attacked called used successful
13  historically  made seen taken able talked described at

In [82]:
model.transformer.wte.weight.shape

torch.Size([50257, 1280])